### Objective: 

Part1
Find the Best restaurant 
features to be considered - rating & check-ins.
Find out the best & print in descending order

Part2
Sentiment Analysis using review

Writing down thoughts on how to approach. I need to find the best restaurant using only two features at the moment
Feature 1 is Ratings 
Feature 2 is check-ins

For ratings we have the following available which is pulled under user_json
For checkins we have checkin_json for which the data is a bit complicated, but we will have to seperate. 

Lets look at the data tables below

In [310]:
import pandas as pd
import numpy as np
import json

In [311]:
user_json = pd.read_json(r'data/yelp_training_set_user.json',lines=True)
review_json = pd.read_json(r'data/yelp_training_set_review.json',lines=True)
checkin_json = pd.read_json(r'data/yelp_training_set_checkin.json',lines=True)
business_json = pd.read_json(r'data/yelp_training_set_business.json',lines=True)

In [312]:
user_json.head() #We would need to seperate votes from dict to a dataframe and also replace name as index

,average_stars,name,review_count,type,user_id,votes
0,5.0,Jim,6,user,CR2y7yEm4X035ZMzrTtN9Q,"{u'funny': 0, u'useful': 7, u'cool': 0}"
1,1.0,Kelle,2,user,_9GXoHhdxc30ujPaQwh6Ew,"{u'funny': 0, u'useful': 1, u'cool': 0}"
2,5.0,Stephanie,2,user,8mM-nqxjg6pT04kwcjMbsw,"{u'funny': 0, u'useful': 1, u'cool': 0}"
3,5.0,T,2,user,Ch6CdTR2IVaVANr-RglMOg,"{u'funny': 0, u'useful': 2, u'cool': 0}"
4,1.0,Beth,1,user,NZrLmHRyiHmyT1JrfzkCOA,"{u'funny': 0, u'useful': 0, u'cool': 0}"


In [313]:
user_votes= user_json['votes']
#As you can see these are in a dict format

In [314]:
user_df = pd.Series(user_votes)
user_df
#As you can see we have now converted this into a series before transf into a dataframe. 

0             {u'funny': 0, u'useful': 7, u'cool': 0}
1             {u'funny': 0, u'useful': 1, u'cool': 0}
2             {u'funny': 0, u'useful': 1, u'cool': 0}
3             {u'funny': 0, u'useful': 2, u'cool': 0}
4             {u'funny': 0, u'useful': 0, u'cool': 0}
5          {u'funny': 30, u'useful': 45, u'cool': 36}
6         {u'funny': 28, u'useful': 130, u'cool': 31}
7             {u'funny': 1, u'useful': 0, u'cool': 1}
8             {u'funny': 0, u'useful': 3, u'cool': 2}
9            {u'funny': 5, u'useful': 24, u'cool': 9}
10           {u'funny': 3, u'useful': 16, u'cool': 6}
11            {u'funny': 0, u'useful': 5, u'cool': 1}
12        {u'funny': 50, u'useful': 109, u'cool': 49}
13            {u'funny': 1, u'useful': 3, u'cool': 1}
14            {u'funny': 0, u'useful': 0, u'cool': 0}
15            {u'funny': 6, u'useful': 4, u'cool': 1}
16            {u'funny': 3, u'useful': 7, u'cool': 3}
17           {u'funny': 2, u'useful': 24, u'cool': 0}
18           {u'funny': 0, u

In [315]:
user_votes_df = user_df.apply(pd.Series)
user_votes_df.tail()
#After 50 tries finally this piece of shit got converted into a dataframe! Phew!

,cool,funny,useful
43868,47,72,117
43869,3,2,6
43870,0,0,0
43871,6,3,5
43872,2,1,4


In [316]:
names=user_json['name']
names.tail()

43868      Geoff
43869    Matthew
43870      tanya
43871      Brian
43872    Lindsey
Name: name, dtype: object

In [317]:
user_ids = user_json['user_id']
user_ids.tail()

43868    SG56r_eAdyK9aad9H8dT4g
43869    brlWQgZoM0_rF3QmmrA99g
43870    bbGCiK43lyi6oqLGNHLMaQ
43871    -iUnAEpltJi0MCjmWrPu9w
43872    aTofoitZDS2l_DoYrmQomA
Name: user_id, dtype: object

In [318]:
combined = [user_ids,names,user_votes_df] #We save this as a list

In [319]:
combined #As you can see it is saved as three lists one below each other. We will concat these mofos. 

[0        CR2y7yEm4X035ZMzrTtN9Q
 1        _9GXoHhdxc30ujPaQwh6Ew
 2        8mM-nqxjg6pT04kwcjMbsw
 3        Ch6CdTR2IVaVANr-RglMOg
 4        NZrLmHRyiHmyT1JrfzkCOA
 5        mWx5Sxt_dx-sYBZg6RgJHQ
 6        hryUDaRk7FLuDAYui2oldw
 7        2t6fZNLtiqsihVmeO7zggg
 8        mn6F-eP5WU37b-iLTop2mQ
 9        myXq7PFXkD_yfXT580SXMw
 10       EMpFiVyiaMS58XsLZdS6vA
 11       74Y-B-8MJhshV7aYSQ93lQ
 12       34ir_XnUsrqserEuSZdGQA
 13       1Aq2NzZGREMRdAkw55V7dg
 14       B7XCvzvcriFc8xfIA-Wg4A
 15       1aoDw1jVm5UQrVacWg0vnw
 16       iH6y1T5AZZxaKpSNmvjGWw
 17       2tyadgcxcaEu5LWgLdwLAQ
 18       mc2glxrNAZuPnCudIaoK4Q
 19       LKFo-Z93fhHHF-CkXoXvLw
 20       09ZCt5inrt35zuJVtxBB5A
 21       aSwY5VAO1S0uZASZlZgB3A
 22       oh3BckWd5WXSANfGlnjErQ
 23       y5YxcvgbQX_uVW0UtHWKxA
 24       Vu29H1sy1poRld4LIL2Jqw
 25       iK7I2HQ8kiyms_XuENr55w
 26       QlSVps_q61IThBo489_Zyw
 27       mzenirXt-7sZR2RhkhvLDQ
 28       qMCT8OblN3finTvXcba-oA
 29       jy94hgtLcuD0BbZ7o8KCYQ
          

In [320]:
user_all = pd.concat(combined, axis =1)

In [321]:
user_all.head() 
#But before I proceed to stack, I might want to delete the index column first

,user_id,name,cool,funny,useful
0,CR2y7yEm4X035ZMzrTtN9Q,Jim,0,0,7
1,_9GXoHhdxc30ujPaQwh6Ew,Kelle,0,0,1
2,8mM-nqxjg6pT04kwcjMbsw,Stephanie,0,0,1
3,Ch6CdTR2IVaVANr-RglMOg,T,0,0,2
4,NZrLmHRyiHmyT1JrfzkCOA,Beth,0,0,0


In [322]:
user_all_final = user_all.set_index('name', drop=True, append=False, inplace=False, verify_integrity=False)
user_all_final.head() #We are assuming at this stage that no reviewier is repeated. Even if repeated, we can solve later. 

,user_id,cool,funny,useful
name,,,,
Jim,CR2y7yEm4X035ZMzrTtN9Q,0,0,7
Kelle,_9GXoHhdxc30ujPaQwh6Ew,0,0,1
Stephanie,8mM-nqxjg6pT04kwcjMbsw,0,0,1
T,Ch6CdTR2IVaVANr-RglMOg,0,0,2
Beth,NZrLmHRyiHmyT1JrfzkCOA,0,0,0


In [323]:
#Now that user rating data is ready, let's clean up the checkin data ('GSA')

checkin

{
'type': 'checkin',
'business_id': (encrypted business id),
'checkin_info': {

'0-0': (number of checkins from 00:00 to 01:00 on all Sundays), 

'1-0': (number of checkins from 01:00 to 02:00 on all Sundays), 

... 

'14-4': (number of checkins from 14:00 to 15:00 on all Thursdays), 

... 

'23-6': (number of checkins from 23:00 to 00:00 on all Saturdays)

} # if there was no checkin for a hour-day block it will not be in the dict
}

In [324]:
#Pulling out the checkin_info column as its pretty much unreadable. Hence lets clean this part up. 
checkin_json.head()

,business_id,checkin_info,type
0,KO9CpaSPOoqm0iCWm5scmg,"{u'11-3': 17, u'10-1': 4, u'15-0': 2, u'15-3':...",checkin
1,oRqBAYtcBYZHXA7G8FlPaA,"{u'0-5': 1, u'2-6': 2, u'2-5': 3, u'3-6': 1, u...",checkin
2,6cy2C9aBXUwkrh4bY1DApw,"{u'8-5': 1, u'20-6': 1, u'15-3': 1, u'18-5': 1...",checkin
3,D0IB17N66FiyYDCzTlAI4A,"{u'17-1': 1, u'10-5': 2, u'15-1': 1, u'20-0': ...",checkin
4,HLQGo3EaYVvAv22bONGkIw,"{u'16-2': 1, u'14-5': 1, u'12-5': 2, u'15-4': ...",checkin


In [325]:
checkin_info=checkin_json['checkin_info']
checkin_info.tail()

8277    {u'19-4': 1, u'13-4': 1, u'11-5': 1, u'17-2': ...
8278    {u'5-0': 1, u'7-3': 2, u'7-2': 1, u'7-1': 1, u...
8279    {u'15-0': 1, u'15-2': 1, u'15-5': 1, u'15-6': ...
8280    {u'22-6': 4, u'22-5': 20, u'22-4': 17, u'22-3'...
8281    {u'17-6': 3, u'15-1': 1, u'15-0': 1, u'17-4': ...
Name: checkin_info, dtype: object

In [326]:
#Now we need to convert this into a dataframe using apply method. 
checkin_df = checkin_info.apply(pd.Series)

In [327]:
checkin_df.tail()

,0-0,0-1,0-2,0-3,0-4,0-5,0-6,1-0,1-1,1-2,...,8-4,8-5,8-6,9-0,9-1,9-2,9-3,9-4,9-5,9-6
8277,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8278,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8279,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN
8280,5.0,6.0,9.0,2.0,4.0,11.0,15.0,1.0,1.0,1.0,...,NaN,1.0,NaN,3.0,1.0,1.0,NaN,NaN,1.0,NaN
8281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,1.0,2.0,1.0,NaN,NaN,NaN


In [328]:
#Above data looks like it could use some fillna instead of having NaN. That way we can aggregate by days also 
#It's better to rename these columns as well instead of trying to infer from numbers what they mean
a=checkin_df.fillna(0)
a

,0-0,0-1,0-2,0-3,0-4,0-5,0-6,1-0,1-1,1-2,...,8-4,8-5,8-6,9-0,9-1,9-2,9-3,9-4,9-5,9-6
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,4.0
1,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,1.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [329]:
b=checkin_json['business_id']

In [330]:
c=[b,a]

In [331]:
final_checkin_data=pd.concat(c,axis=1)

In [332]:
final_checkin_data.head()

,business_id,0-0,0-1,0-2,0-3,0-4,0-5,0-6,1-0,1-1,...,8-4,8-5,8-6,9-0,9-1,9-2,9-3,9-4,9-5,9-6
0,KO9CpaSPOoqm0iCWm5scmg,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,4.0
1,oRqBAYtcBYZHXA7G8FlPaA,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6cy2C9aBXUwkrh4bY1DApw,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,D0IB17N66FiyYDCzTlAI4A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,HLQGo3EaYVvAv22bONGkIw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### x-x-x-x-x-x-x-x

In [333]:
#Check-ins data, I need to merge it to business_json as business id is common. Busines_json will form the 
#basis for the big happy table which can be used for all analysis. 
business_json.head()

,business_id,categories,city,full_address,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type
0,rncjoVoEFUJGCUoC1JgnUA,"[Accountants, Professional Services, Tax Servi...",Peoria,"8466 W Peoria Ave\nSte 6\nPeoria, AZ 85345",33.581867,-112.241596,Peoria Income Tax Service,[],True,3,5.0,AZ,business
1,0FNFSzCFP_rGUoJx8W7tJg,"[Sporting Goods, Bikes, Shopping]",Phoenix,"2149 W Wood Dr\nPhoenix, AZ 85029",33.604054,-112.105933,Bike Doctor,[],True,5,5.0,AZ,business
2,3f_lyB6vFK48ukH6ScvLHg,[],Phoenix,"1134 N Central Ave\nPhoenix, AZ 85004",33.460526,-112.073933,Valley Permaculture Alliance,[],True,4,5.0,AZ,business
3,usAsSV36QmUej8--yvN-dg,"[Food, Grocery]",Phoenix,"845 W Southern Ave\nPhoenix, AZ 85041",33.392210,-112.085377,Food City,[],True,5,3.5,AZ,business
4,PzOqRohWw7F7YEPBz6AubA,"[Food, Bagels, Delis, Restaurants]",Glendale Az,"6520 W Happy Valley Rd\nSte 101\nGlendale Az, ...",33.712797,-112.200264,Hot Bagels & Deli,[],True,14,3.5,AZ,business


In [334]:
review_json.head() #Here I can attach the user_all_final data to review since user_id is common. Then taking business id,
#I can attach this to the business _json table and make one big happy table which we can sort. 


,business_id,date,review_id,stars,text,type,user_id,votes
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,"{u'funny': 0, u'useful': 5, u'cool': 2}"
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,"{u'funny': 0, u'useful': 0, u'cool': 0}"
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,"{u'funny': 0, u'useful': 1, u'cool': 0}"
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,"{u'funny': 0, u'useful': 2, u'cool': 1}"
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,"{u'funny': 0, u'useful': 0, u'cool': 0}"


### x-x-x-x-x-x-x

In [335]:
business_json.head()

,business_id,categories,city,full_address,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type
0,rncjoVoEFUJGCUoC1JgnUA,"[Accountants, Professional Services, Tax Servi...",Peoria,"8466 W Peoria Ave\nSte 6\nPeoria, AZ 85345",33.581867,-112.241596,Peoria Income Tax Service,[],True,3,5.0,AZ,business
1,0FNFSzCFP_rGUoJx8W7tJg,"[Sporting Goods, Bikes, Shopping]",Phoenix,"2149 W Wood Dr\nPhoenix, AZ 85029",33.604054,-112.105933,Bike Doctor,[],True,5,5.0,AZ,business
2,3f_lyB6vFK48ukH6ScvLHg,[],Phoenix,"1134 N Central Ave\nPhoenix, AZ 85004",33.460526,-112.073933,Valley Permaculture Alliance,[],True,4,5.0,AZ,business
3,usAsSV36QmUej8--yvN-dg,"[Food, Grocery]",Phoenix,"845 W Southern Ave\nPhoenix, AZ 85041",33.392210,-112.085377,Food City,[],True,5,3.5,AZ,business
4,PzOqRohWw7F7YEPBz6AubA,"[Food, Bagels, Delis, Restaurants]",Glendale Az,"6520 W Happy Valley Rd\nSte 101\nGlendale Az, ...",33.712797,-112.200264,Hot Bagels & Deli,[],True,14,3.5,AZ,business


In [336]:
e=business_json.merge(final_checkin_data,how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False)
e

,business_id,categories,city,full_address,latitude,longitude,name,neighborhoods,open,review_count,...,8-4,8-5,8-6,9-0,9-1,9-2,9-3,9-4,9-5,9-6
0,usAsSV36QmUej8--yvN-dg,"[Food, Grocery]",Phoenix,"845 W Southern Ave\nPhoenix, AZ 85041",33.392210,-112.085377,Food City,[],True,5,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,PzOqRohWw7F7YEPBz6AubA,"[Food, Bagels, Delis, Restaurants]",Glendale Az,"6520 W Happy Valley Rd\nSte 101\nGlendale Az, ...",33.712797,-112.200264,Hot Bagels & Deli,[],True,14,...,1.0,0.0,4.0,3.0,1.0,1.0,0.0,0.0,9.0,6.0
2,FrBCYtCS_jydDjg1KsIgWQ,"[Music & DVDs, Books, Mags, Music & Video, Vin...",Phoenix,"1850 W Camelback Rd\nSte 1\nPhoenix, AZ 85015",33.509658,-112.098750,Zia Record Exchange,[],True,21,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,yaXAD-Mv2K2PEZobmqjIYA,"[Event Planning & Services, Venues & Event Spa...",Phoenix,"3415 W Northern Ave\nPhoenix, AZ 85051",33.551903,-112.132866,Reflections Bingo,[],True,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,o3ehs4ZEdsizbJyB9_j7uQ,"[Art Schools, Specialty Schools, Shopping, Jew...",Scottsdale,"6938 E 1st St\nScottsdale, AZ 85251",33.492195,-111.931299,Jam,[],True,14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,qarobAbxGSHI7ygf1f7a_Q,"[Sandwiches, Restaurants]",Gilbert,"891 E Baseline Rd\nSuite 102\nGilbert, AZ 85233",33.378839,-111.812007,Jersey Mike's Subs,[],True,10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,C1xciP65n4t4R1Q4h1zOHw,"[Music & DVDs, Books, Mags, Music & Video, Mus...",Scottsdale,"8860 E Chaparral Rd\nSte 100\nScottsdale, AZ 8...",33.513519,-111.888789,Fender Musical Instruments Corporation,[],True,4,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,gA5CuBxF-0CnOpGnryWJdQ,"[Mexican, Restaurants]",Phoenix,"519 W Thomas Rd\nPhoenix, AZ 85013",33.480105,-112.081361,La Paloma Mexican Food,[],True,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8,acaBJcFEKPmmSDIO6c-ZGQ,"[Food, Grocery]",Goodyear,"2075 N Pebble Creek Pkwy\nGoodyear, AZ 85395",33.468815,-112.392022,Basha's,[],True,3,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9,JxVGJ9Nly2FFIs_WpJvkug,"[Pizza, Restaurants]",Scottsdale,"14418 N Scottsdale Rd\nSuite 181\nScottsdale, ...",33.617459,-111.926272,Sauce,[],True,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [337]:
e.tail()

,business_id,categories,city,full_address,latitude,longitude,name,neighborhoods,open,review_count,...,8-4,8-5,8-6,9-0,9-1,9-2,9-3,9-4,9-5,9-6
8277,542yzNlAhZK9qk6Ibqd06w,"[Automotive, Auto Parts & Supplies]",Phoenix,"3107 E Indian School Road\nPhoenix, AZ 85016",33.495196,-112.014616,Checker Auto Parts,[],True,3,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
8278,TCIVUf1frqLehZbaUnuQzQ,"[Mexican, Restaurants]",Tempe,"1158 W Washington St\nSte 101\nTempe, AZ 85281",33.442375,-111.955643,Filiberto's Mexican Food,[],True,18,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
8279,F3tqTcfKnljJcSyyqN0bbw,"[Mexican, Restaurants]",Scottsdale,"8489 E McDonald Dr\nScottsdale, AZ 85250",33.523891,-111.897904,Los Favoritos Taco Shop,[],True,25,...,0.0,1.0,3.0,0.0,5.0,4.0,2.0,2.0,3.0,7.0
8280,diW9KhKuhBCblBdnAkUkpQ,"[Food, Grocery]",Phoenix,"550 E Bell Rd\nPhoenix, AZ 85022",33.640369,-112.067004,Safeway,[],True,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8281,tQTqlMW0sODNbe188ah1Yg,"[Greek, Mediterranean, Restaurants]",Peoria,"8378 W Thunderbird Rd\nPeoria, AZ 85381",33.611202,-112.240062,Gyros Palace,[],True,23,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [338]:
f=pd.merge(e,review_json,on='business_id', how='outer')
f.tail()
#So I felt that this was incorrectly tabulated initially but then again it is correct. Each business id repeats as 
# ...multiple reviews appear against it. 

,business_id,categories,city,full_address,latitude,longitude,name,neighborhoods,open,review_count,...,9-4,9-5,9-6,date,review_id,stars_y,text,type_y,user_id,votes
229902,OBbZsvsdz5dKedj7xGBAEw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2012-11-25,ZrFbsCyr_F3ucfIRww69ag,4,Everything I need for an improptu Carne Asada ...,review,bvvLS4Vz5yFuNvGKBV3-1g,"{u'funny': 0, u'useful': 1, u'cool': 0}"
229903,OBbZsvsdz5dKedj7xGBAEw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2012-12-02,vUAymdi3JLEdjCilZvif-w,4,Excellent carniceria in Peoria. I agree with o...,review,-kIvLyWpY17aRa0vPp-RmA,"{u'funny': 0, u'useful': 1, u'cool': 1}"
229904,SeCVec3f91bEdosAILE4JA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2007-12-11,SgUeAdo1FgJ1crq-SH0vIw,5,I think I have died and gone to DVF heaven. (...,review,qASPib1Z8ft8e96dtbh66w,"{u'funny': 2, u'useful': 5, u'cool': 5}"
229905,SeCVec3f91bEdosAILE4JA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2007-09-07,i0uwwifLtJPkfbdSrlPvwg,5,What a dream! This place is stocked to the bri...,review,9FbgqZ5u9WTLAs28nNafIQ,"{u'funny': 1, u'useful': 3, u'cool': 2}"
229906,SeCVec3f91bEdosAILE4JA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2008-01-26,0f-NizYApb8ZbY6oD2NGXw,5,Awesome boutique! The selection is amazing...l...,review,jqVeTSin5GeRm0ceSg-PBA,"{u'funny': 0, u'useful': 2, u'cool': 1}"


In [339]:
#Now we need to delete the votes column
g=f.drop('votes', axis=1)

In [340]:
g.tail()

,business_id,categories,city,full_address,latitude,longitude,name,neighborhoods,open,review_count,...,9-3,9-4,9-5,9-6,date,review_id,stars_y,text,type_y,user_id
229902,OBbZsvsdz5dKedj7xGBAEw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2012-11-25,ZrFbsCyr_F3ucfIRww69ag,4,Everything I need for an improptu Carne Asada ...,review,bvvLS4Vz5yFuNvGKBV3-1g
229903,OBbZsvsdz5dKedj7xGBAEw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2012-12-02,vUAymdi3JLEdjCilZvif-w,4,Excellent carniceria in Peoria. I agree with o...,review,-kIvLyWpY17aRa0vPp-RmA
229904,SeCVec3f91bEdosAILE4JA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2007-12-11,SgUeAdo1FgJ1crq-SH0vIw,5,I think I have died and gone to DVF heaven. (...,review,qASPib1Z8ft8e96dtbh66w
229905,SeCVec3f91bEdosAILE4JA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2007-09-07,i0uwwifLtJPkfbdSrlPvwg,5,What a dream! This place is stocked to the bri...,review,9FbgqZ5u9WTLAs28nNafIQ
229906,SeCVec3f91bEdosAILE4JA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2008-01-26,0f-NizYApb8ZbY6oD2NGXw,5,Awesome boutique! The selection is amazing...l...,review,jqVeTSin5GeRm0ceSg-PBA


In [341]:
#Now let's add user review attributes such as cool, funny and useful to the main table g. 
h=pd.merge(g,user_all,on='user_id', how='outer')

In [342]:
h.head()

,business_id,categories,city,full_address,latitude,longitude,name_x,neighborhoods,open,review_count,...,date,review_id,stars_y,text,type_y,user_id,name_y,cool,funny,useful
0,usAsSV36QmUej8--yvN-dg,"[Food, Grocery]",Phoenix,"845 W Southern Ave\nPhoenix, AZ 85041",33.392210,-112.085377,Food City,[],True,5.0,...,2012-12-20,Xa-RIEiCLibHDHrWXbmfcw,4,"I go here and get my fruits and veggies,and th...",review,WgpyvVqYAc3FOvk-G-Erwg,Erik,6.0,4.0,7.0
1,0dIChGWBs9ueUXl0hhkw2w,"[Greek, Mediterranean, Sandwiches, Restaurants]",Phoenix,"4041 N 15th Ave\nPhoenix, AZ 85015",33.494367,-112.091252,Crazy Jim's Restaurant,[],True,79.0,...,2010-10-02,oGtSU2M-vv8Hh5DMCXJWvQ,4,My girlfriend and I were hungry and wanted to ...,review,WgpyvVqYAc3FOvk-G-Erwg,Erik,6.0,4.0,7.0
2,eQ9I23C8O9ruHIaupxp40g,"[Barbers, Beauty & Spas]",Phoenix,"1712 E Broadway Road\nPhoenix, AZ 85040",33.407064,-112.044479,Fade Tite Barbershop,[],True,3.0,...,2012-04-20,lgHMjCMWdp6kAqlPlFTsQw,4,I would have to say this shop has guys that g...,review,WgpyvVqYAc3FOvk-G-Erwg,Erik,6.0,4.0,7.0
3,P7wKlbSRsoRrfH3DFOY08A,"[Breakfast & Brunch, Restaurants]",Phoenix,"2508 N Central Ave\nPhoenix, AZ 85004",33.475498,-112.074160,International House of Pancakes,[],True,23.0,...,2012-01-22,-kwiYK-Xxvc6I6f03lYK1g,3,"It's Ihop, good when nothing else is open and ...",review,WgpyvVqYAc3FOvk-G-Erwg,Erik,6.0,4.0,7.0
4,OFBJtqWGSx6n6CchtVmoRw,"[Burgers, Bars, Nightlife, Restaurants]",Phoenix,"3146 E Camelback Rd\nPhoenix, AZ 85016",33.510040,-112.014641,Delux,[],True,502.0,...,2012-01-22,4J3Q-b4bvOyB8-wpyuRO7Q,5,My girfriend and I have never had a bad experi...,review,WgpyvVqYAc3FOvk-G-Erwg,Erik,6.0,4.0,7.0


In [350]:
#Crosscheck threw up Nan in a few rows. Need to re-look at this. 
business_json[business_json['business_id']=='NlD18mwNRjNPzH8hs7abPQ']

,business_id,categories,city,full_address,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type
5254,NlD18mwNRjNPzH8hs7abPQ,"[Home Services, Pool Cleaners]",Phoenix,"519 W Lone Cactus Dr\nSte 101\nPhoenix, AZ 85027",33.680405,-112.081124,Splish Splash Pool Service and Repair,[],True,4,3.5,AZ,business


#### Let's first pull out top restaurants via reviews first. We can have a niche relevant sample at hand. 

    * First lets try to add a total count column per establishment. For this we can use groupby. 
    * Let's take the count of reviews first. 

In [351]:
review_json

,business_id,date,review_id,stars,text,type,user_id,votes
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,"{u'funny': 0, u'useful': 5, u'cool': 2}"
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,"{u'funny': 0, u'useful': 0, u'cool': 0}"
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,"{u'funny': 0, u'useful': 1, u'cool': 0}"
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,"{u'funny': 0, u'useful': 2, u'cool': 1}"
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,"{u'funny': 0, u'useful': 0, u'cool': 0}"
5,-yxfBYGB6SEqszmxJxd97A,2007-12-13,m2CKSsepBCoRYWxiRUsxAg,4,"Quiessence is, simply put, beautiful. Full wi...",review,sqYN3lNgvPbPCTRsMFu27g,"{u'funny': 1, u'useful': 3, u'cool': 4}"
6,zp713qNhx8d9KCJJnrw1xA,2010-02-12,riFQ3vxNpP4rWLk_CSri2A,5,Drop what you're doing and drive here. After I...,review,wFweIWhv2fREZV_dYkz_1g,"{u'funny': 4, u'useful': 7, u'cool': 7}"
7,hW0Ne_HTHEAgGF1rAdmR-g,2012-07-12,JL7GXJ9u4YMx7Rzs05NfiQ,4,"Luckily, I didn't have to travel far to make m...",review,1ieuYcKS7zeAv_U15AB13A,"{u'funny': 0, u'useful': 1, u'cool': 0}"
8,wNUea3IXZWD63bbOQaOH-g,2012-08-17,XtnfnYmnJYi71yIuGsXIUA,4,Definitely come for Happy hour! Prices are ama...,review,Vh_DlizgGhSqQh4qfZ2h6A,"{u'funny': 0, u'useful': 0, u'cool': 0}"
9,nMHhuYan8e3cONo3PornJA,2010-08-11,jJAIXA46pU1swYyRCdfXtQ,5,Nobuo shows his unique talents with everything...,review,sUNkXg8-KFtCMQDV6zRzQg,"{u'funny': 0, u'useful': 1, u'cool': 0}"


In [352]:
Grouped=pd.DataFrame(review_json.groupby('business_id')['text'].count())

In [353]:
Grouped_sorted = Grouped.sort_values('text',ascending=False)

In [354]:
Grouped_sorted.tail()

,text
business_id,
UKgvVkQ9WldngU7KHt8MAQ,3
UJhIuwQniicO_SbfB1K5wg,3
AqdFUHo_is8NfBh_n7tlHg,3
AqpMmUEyxLF_zWAuzXlSWw,3
fXtPvVb8vbWvAa_mRjq2ig,3


In [355]:
Grouped_sorted1 = Grouped_sorted[Grouped_sorted['text'] > 300]

In [356]:
Grouped_sorted1.head()

,text
business_id,
hW0Ne_HTHEAgGF1rAdmR-g,844
VVeogjZya58oiTxK7qUjAQ,794
JokKtdXU7zXHcr20Lrk29A,731
ntN85eu27C04nwyPa8IHtw,679
EWMwV5V9BxNs_U6nNVMeqw,645


In [357]:
top_restro=Grouped_sorted1.rename(columns = {'text':'textreview_count'})

In [358]:
top_restro.tail()

,textreview_count
business_id,
Xq9tkiHhyN_aBFswFeGLvA,307
AkOruz5CrCxUmXe1p_WoRg,306
zp713qNhx8d9KCJJnrw1xA,303
cp39VURFby4A6u9m2It8jg,302
V6BYjHM9SnXscfkln3hCdA,301


In [359]:
#Now that we have top restaurants, let's merge all other tables based on these. 
business_json.tail()

,business_id,categories,city,full_address,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type
11532,TCIVUf1frqLehZbaUnuQzQ,"[Mexican, Restaurants]",Tempe,"1158 W Washington St\nSte 101\nTempe, AZ 85281",33.442375,-111.955643,Filiberto's Mexican Food,[],True,18,3.0,AZ,business
11533,F3tqTcfKnljJcSyyqN0bbw,"[Mexican, Restaurants]",Scottsdale,"8489 E McDonald Dr\nScottsdale, AZ 85250",33.523891,-111.897904,Los Favoritos Taco Shop,[],True,25,3.5,AZ,business
11534,diW9KhKuhBCblBdnAkUkpQ,"[Food, Grocery]",Phoenix,"550 E Bell Rd\nPhoenix, AZ 85022",33.640369,-112.067004,Safeway,[],True,3,4.5,AZ,business
11535,tQTqlMW0sODNbe188ah1Yg,"[Greek, Mediterranean, Restaurants]",Peoria,"8378 W Thunderbird Rd\nPeoria, AZ 85381",33.611202,-112.240062,Gyros Palace,[],True,23,3.5,AZ,business
11536,6yVe_iet5qD7SzVMhRYYog,"[Print Media, Mass Media]",Scottsdale,"4253 N scottsdale Rd\n#200\nScottsdale, AZ 85251",33.498170,-111.925638,944 Media,[],True,11,2.5,AZ,business


In [360]:
reindexed = business_json.set_index('business_id', drop=True, append=False, inplace=False, verify_integrity=False)
reindexed

,categories,city,full_address,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type
business_id,,,,,,,,,,,,
rncjoVoEFUJGCUoC1JgnUA,"[Accountants, Professional Services, Tax Servi...",Peoria,"8466 W Peoria Ave\nSte 6\nPeoria, AZ 85345",33.581867,-112.241596,Peoria Income Tax Service,[],True,3,5.0,AZ,business
0FNFSzCFP_rGUoJx8W7tJg,"[Sporting Goods, Bikes, Shopping]",Phoenix,"2149 W Wood Dr\nPhoenix, AZ 85029",33.604054,-112.105933,Bike Doctor,[],True,5,5.0,AZ,business
3f_lyB6vFK48ukH6ScvLHg,[],Phoenix,"1134 N Central Ave\nPhoenix, AZ 85004",33.460526,-112.073933,Valley Permaculture Alliance,[],True,4,5.0,AZ,business
usAsSV36QmUej8--yvN-dg,"[Food, Grocery]",Phoenix,"845 W Southern Ave\nPhoenix, AZ 85041",33.392210,-112.085377,Food City,[],True,5,3.5,AZ,business
PzOqRohWw7F7YEPBz6AubA,"[Food, Bagels, Delis, Restaurants]",Glendale Az,"6520 W Happy Valley Rd\nSte 101\nGlendale Az, ...",33.712797,-112.200264,Hot Bagels & Deli,[],True,14,3.5,AZ,business
gtQzAiy7D-dPU8WzT3jX3Q,"[Women's Clothing, Fashion, Shopping]",Glendale,The Americana at Brand\n869 Americana Way\nGle...,33.607766,-112.481632,Barney's New York Co-op,[],True,6,4.5,CA,business
FrBCYtCS_jydDjg1KsIgWQ,"[Music & DVDs, Books, Mags, Music & Video, Vin...",Phoenix,"1850 W Camelback Rd\nSte 1\nPhoenix, AZ 85015",33.509658,-112.098750,Zia Record Exchange,[],True,21,3.5,AZ,business
yaXAD-Mv2K2PEZobmqjIYA,"[Event Planning & Services, Venues & Event Spa...",Phoenix,"3415 W Northern Ave\nPhoenix, AZ 85051",33.551903,-112.132866,Reflections Bingo,[],True,4,5.0,AZ,business
o3ehs4ZEdsizbJyB9_j7uQ,"[Art Schools, Specialty Schools, Shopping, Jew...",Scottsdale,"6938 E 1st St\nScottsdale, AZ 85251",33.492195,-111.931299,Jam,[],True,14,5.0,AZ,business


In [361]:
big_table=top_restro.merge(reindexed, how='inner', left_index=True, right_index=True)
#DataFrame.merge(right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False)

In [362]:
sorted_BT = big_table.sort_values('textreview_count', ascending=False)
#sort_values(self, by, axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [363]:
sorted_BT.head()

,textreview_count,categories,city,full_address,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type
business_id,,,,,,,,,,,,,
hW0Ne_HTHEAgGF1rAdmR-g,844,"[Hotels & Travel, Airports]",Phoenix,"3400 E Sky Harbor Blvd\nPhoenix, AZ 85034",33.434750,-112.006440,Phoenix Sky Harbor International Airport,[],True,862,3.0,AZ,business
VVeogjZya58oiTxK7qUjAQ,794,"[Pizza, Restaurants]",Phoenix,"623 E Adams St\nPhoenix, AZ 85004",33.449233,-112.065458,Pizzeria Bianco,[],True,803,4.0,AZ,business
JokKtdXU7zXHcr20Lrk29A,731,"[Bars, Food, Breweries, Pubs, Nightlife, Ameri...",Tempe,"1340 E 8th St\nSte 104\nTempe, AZ 85281",33.419451,-111.915926,Four Peaks Brewing Co,[],True,735,4.5,AZ,business
ntN85eu27C04nwyPa8IHtw,679,"[Breakfast & Brunch, Restaurants]",Phoenix,"801 N 1st St\nPhoenix, AZ 85004",33.456696,-112.072327,Matt's Big Breakfast,[],True,689,4.0,AZ,business
EWMwV5V9BxNs_U6nNVMeqw,645,"[Bars, Mediterranean, Nightlife, Lounges, Amer...",Phoenix,"3815 N Central Ave\nPhoenix, AZ 85012",33.491645,-112.073196,FEZ,[],True,652,4.0,AZ,business


In [364]:
BTcategories=sorted_BT['categories']
pd.DataFrame(BTcategories).tail()

,categories
business_id,
Xq9tkiHhyN_aBFswFeGLvA,"[American (New), Restaurants]"
AkOruz5CrCxUmXe1p_WoRg,"[Asian Fusion, Italian, Japanese, Restaurants]"
zp713qNhx8d9KCJJnrw1xA,"[Mexican, Restaurants]"
cp39VURFby4A6u9m2It8jg,"[Burgers, Restaurants]"
V6BYjHM9SnXscfkln3hCdA,"[Breakfast & Brunch, American (Traditional), R..."


In [365]:
BT_categories = BTcategories.apply(pd.Series)

In [366]:
BT_categories.head()

#<ALL OF THESE ARE FOOD RELATED ESTABLISHMENTS IRONICALLY>
# Hence we can proceed with attaching other table details using business_id as on=

,0,1,2,3,4,5,6
business_id,,,,,,,
hW0Ne_HTHEAgGF1rAdmR-g,Hotels & Travel,Airports,NaN,NaN,NaN,NaN,NaN
VVeogjZya58oiTxK7qUjAQ,Pizza,Restaurants,NaN,NaN,NaN,NaN,NaN
JokKtdXU7zXHcr20Lrk29A,Bars,Food,Breweries,Pubs,Nightlife,American (New),Restaurants
ntN85eu27C04nwyPa8IHtw,Breakfast & Brunch,Restaurants,NaN,NaN,NaN,NaN,NaN
EWMwV5V9BxNs_U6nNVMeqw,Bars,Mediterranean,Nightlife,Lounges,American (New),Restaurants,NaN


In [367]:
sorted_BT.tail()

,textreview_count,categories,city,full_address,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type
business_id,,,,,,,,,,,,,
Xq9tkiHhyN_aBFswFeGLvA,307,"[American (New), Restaurants]",Phoenix,"2 E Jefferson St\nSte 150\nPhoenix, AZ 85004",33.447173,-112.073781,The Arrogant Butcher,[],True,321,4.0,AZ,business
AkOruz5CrCxUmXe1p_WoRg,306,"[Asian Fusion, Italian, Japanese, Restaurants]",Phoenix,"914 E Camelback Rd\nPhoenix, AZ 85014",33.509337,-112.060818,CherryBlossom Noodle Cafe,[],True,309,4.0,AZ,business
zp713qNhx8d9KCJJnrw1xA,303,"[Mexican, Restaurants]",Phoenix,"1919 N 16th St\nPhoenix, AZ 85006",33.469132,-112.047512,La Condesa Gourmet Taco Shop,[],True,307,4.0,AZ,business
cp39VURFby4A6u9m2It8jg,302,"[Burgers, Restaurants]",Phoenix,"2502 E Camelback Rd\nSte 127\nPhoenix, AZ 85016",33.509535,-112.027592,Zinburger,[],True,305,3.5,AZ,business
V6BYjHM9SnXscfkln3hCdA,301,"[Breakfast & Brunch, American (Traditional), R...",Scottsdale,"7373 E Scottsdale Mall\nSte 6\nScottsdale, AZ ...",33.492502,-111.924119,Orange Table,[],True,304,4.0,AZ,business


In [368]:
review_json
#final_checkin_data

,business_id,date,review_id,stars,text,type,user_id,votes
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,"{u'funny': 0, u'useful': 5, u'cool': 2}"
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,"{u'funny': 0, u'useful': 0, u'cool': 0}"
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,"{u'funny': 0, u'useful': 1, u'cool': 0}"
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,"{u'funny': 0, u'useful': 2, u'cool': 1}"
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,"{u'funny': 0, u'useful': 0, u'cool': 0}"
5,-yxfBYGB6SEqszmxJxd97A,2007-12-13,m2CKSsepBCoRYWxiRUsxAg,4,"Quiessence is, simply put, beautiful. Full wi...",review,sqYN3lNgvPbPCTRsMFu27g,"{u'funny': 1, u'useful': 3, u'cool': 4}"
6,zp713qNhx8d9KCJJnrw1xA,2010-02-12,riFQ3vxNpP4rWLk_CSri2A,5,Drop what you're doing and drive here. After I...,review,wFweIWhv2fREZV_dYkz_1g,"{u'funny': 4, u'useful': 7, u'cool': 7}"
7,hW0Ne_HTHEAgGF1rAdmR-g,2012-07-12,JL7GXJ9u4YMx7Rzs05NfiQ,4,"Luckily, I didn't have to travel far to make m...",review,1ieuYcKS7zeAv_U15AB13A,"{u'funny': 0, u'useful': 1, u'cool': 0}"
8,wNUea3IXZWD63bbOQaOH-g,2012-08-17,XtnfnYmnJYi71yIuGsXIUA,4,Definitely come for Happy hour! Prices are ama...,review,Vh_DlizgGhSqQh4qfZ2h6A,"{u'funny': 0, u'useful': 0, u'cool': 0}"
9,nMHhuYan8e3cONo3PornJA,2010-08-11,jJAIXA46pU1swYyRCdfXtQ,5,Nobuo shows his unique talents with everything...,review,sUNkXg8-KFtCMQDV6zRzQg,"{u'funny': 0, u'useful': 1, u'cool': 0}"


In [369]:
Nreview_json=review_json.drop('votes', axis=1)

In [371]:
NNReview_json = Nreview_json.set_index('business_id')
#reindexed = business_json.set_index('business_id', drop=True, append=False, inplace=False, verify_integrity=False)

In [373]:
NNReview_json.tail()

,date,review_id,stars,text,type,user_id
business_id,,,,,,
vnffHkFJbmd-J3OaBbK2Eg,2011-04-14,f9JaiNg_FMoPNWxt7MlbZQ,2,I really wanted to like this place because it'...,review,6e7pZofhDuIlD_rX2oYirQ
l5oUrgQ190l8CcN8uzd_pA,2011-01-23,QDWRP1pW5r0huIBAoGmFyg,1,My husband I stayed here for two nights. Of c...,review,dDNfSFT0VApxPmURclX6_g
-EctXOb3B7T177jGYUhjVA,2010-10-11,JmR3yk7JlS1LVVxtIc3xBQ,4,Cool atmosphere. A lot of beers on tap and goo...,review,M5wHt6Odh1k5v0tIjqd8DQ
YQvg0JCGRFUkb6reMMf3Iw,2011-01-18,z5b2p5TbCg0uaIiIe8n62w,3,I have to take a star off for the spotty servi...,review,jopndPrv-H5KW2CfScnw9A
gKxOZvQTTd5hpFa3r5igGQ,2010-08-14,QM1rFJsW-ZJoCHbgsysKaw,5,"So cool, yo.",review,mbRUG6h0Mgb2XIJvscIoMg


In [392]:
BT_1 = sorted_BT.merge(NNReview_json, how='inner', left_index=True, right_index=True)
BT_1
#big_table=top_restro.merge(reindexed, how='inner', left_index=True, right_index=True)

,textreview_count,categories,city,full_address,latitude,longitude,name,neighborhoods,open,review_count,stars_x,state,type_x,date,review_id,stars_y,text,type_y,user_id
business_id,,,,,,,,,,,,,,,,,,,
-sC66z4SO3tR7nFCjfQwuQ,540,"[Mexican, Restaurants]",Phoenix,"401 W Clarendon Ave\nPhoenix, AZ 85013",33.491120,-112.079081,Gallo Blanco Cafe,[],True,549,4.0,AZ,business,2012-12-06,7asKG4Bt_PXDE_5SZ1TxFw,3,"The last three times I have been here, the hos...",review,slp8VQWuW0Dq_azDCAshCw
-sC66z4SO3tR7nFCjfQwuQ,540,"[Mexican, Restaurants]",Phoenix,"401 W Clarendon Ave\nPhoenix, AZ 85013",33.491120,-112.079081,Gallo Blanco Cafe,[],True,549,4.0,AZ,business,2011-01-22,us79WZcTQ5lH_EticmhReA,4,Joined @SMUNCHPhoenix at @GalloBlanco.\n\nFor ...,review,7cR92zkDv4W3kqzii6axvg
-sC66z4SO3tR7nFCjfQwuQ,540,"[Mexican, Restaurants]",Phoenix,"401 W Clarendon Ave\nPhoenix, AZ 85013",33.491120,-112.079081,Gallo Blanco Cafe,[],True,549,4.0,AZ,business,2012-05-28,-cDLdPqZyJwslg500NjwOA,5,Love this place. Great Happy Hour.,review,Dngg6R24eGnEry89Tf926A
-sC66z4SO3tR7nFCjfQwuQ,540,"[Mexican, Restaurants]",Phoenix,"401 W Clarendon Ave\nPhoenix, AZ 85013",33.491120,-112.079081,Gallo Blanco Cafe,[],True,549,4.0,AZ,business,2012-06-21,zqbJKo1GmJ694PQ-M_4y3g,5,Gallo Blanco has the most delicious ceviche I'...,review,yV6rKdVT7E833ffE6UHwRg
-sC66z4SO3tR7nFCjfQwuQ,540,"[Mexican, Restaurants]",Phoenix,"401 W Clarendon Ave\nPhoenix, AZ 85013",33.491120,-112.079081,Gallo Blanco Cafe,[],True,549,4.0,AZ,business,2009-07-30,mB17dzreCHZs4QhnHRAp3w,5,Why am I trying to hide my love...\n\nI'm at G...,review,Yik3_gi3e2F5WsKOFm14QQ
-sC66z4SO3tR7nFCjfQwuQ,540,"[Mexican, Restaurants]",Phoenix,"401 W Clarendon Ave\nPhoenix, AZ 85013",33.491120,-112.079081,Gallo Blanco Cafe,[],True,549,4.0,AZ,business,2012-01-16,GlFyloK36QBo7Qwn1DVarQ,2,After stopping by Gallo Blanco for breakfast o...,review,GJ81EquQfQKvS-eLRdr6aQ
-sC66z4SO3tR7nFCjfQwuQ,540,"[Mexican, Restaurants]",Phoenix,"401 W Clarendon Ave\nPhoenix, AZ 85013",33.491120,-112.079081,Gallo Blanco Cafe,[],True,549,4.0,AZ,business,2010-07-06,88PBEqEtElumaEXXEXb5AQ,5,I think its about time I wrote a formal review...,review,B4K5ruafil9RSDiTIL-1Bw
-sC66z4SO3tR7nFCjfQwuQ,540,"[Mexican, Restaurants]",Phoenix,"401 W Clarendon Ave\nPhoenix, AZ 85013",33.491120,-112.079081,Gallo Blanco Cafe,[],True,549,4.0,AZ,business,2012-10-01,BvXTRXYZtbTunoi59CexUg,4,My roommate and I went here for drinks before ...,review,Ypz7hxOCnrg8Y8vxHJU-sQ
-sC66z4SO3tR7nFCjfQwuQ,540,"[Mexican, Restaurants]",Phoenix,"401 W Clarendon Ave\nPhoenix, AZ 85013",33.491120,-112.079081,Gallo Blanco Cafe,[],True,549,4.0,AZ,business,2012-03-07,rMjOGresCVz2dvJXJl5-MA,4,I love the feel of this cute restaurant! It wa...,review,l5upkLok78p-NQN_3uurrA


In [382]:
#Let's add user data as well (cool, funny..)
#BT_2 = BT_1.merge(user_all, how='left', on='user_id')

In [393]:
BT_user_all = user_all.set_index('user_id')
BT_user_all.tail()

,name,cool,funny,useful
user_id,,,,
SG56r_eAdyK9aad9H8dT4g,Geoff,47,72,117
brlWQgZoM0_rF3QmmrA99g,Matthew,3,2,6
bbGCiK43lyi6oqLGNHLMaQ,tanya,0,0,0
-iUnAEpltJi0MCjmWrPu9w,Brian,6,3,5
aTofoitZDS2l_DoYrmQomA,Lindsey,2,1,4


In [396]:
BT_2 = BT_1.merge(BT_user_all, left_on='user_id', right_index=True)
#df1.join(df2)
#df1.merge(df2, left_on='name', right_index=True)

In [400]:
a.tail()

,0-0,0-1,0-2,0-3,0-4,0-5,0-6,1-0,1-1,1-2,...,8-4,8-5,8-6,9-0,9-1,9-2,9-3,9-4,9-5,9-6
8277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8278,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
8280,5.0,6.0,9.0,2.0,4.0,11.0,15.0,1.0,1.0,1.0,...,0.0,1.0,0.0,3.0,1.0,1.0,0.0,0.0,1.0,0.0
8281,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0


In [464]:
# Can pick out for Sunday and figure which is the most busiest restaurant
final_checkin_data.head()

,0-0,0-1,0-2,0-3,0-4,0-5,0-6,1-0,1-1,1-2,...,8-4,8-5,8-6,9-0,9-1,9-2,9-3,9-4,9-5,9-6
business_id,,,,,,,,,,,,,,,,,,,,,
KO9CpaSPOoqm0iCWm5scmg,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,4.0
oRqBAYtcBYZHXA7G8FlPaA,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6cy2C9aBXUwkrh4bY1DApw,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
D0IB17N66FiyYDCzTlAI4A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HLQGo3EaYVvAv22bONGkIw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [463]:
#If we do not set business_id as the index then we will loose the column when we try endswith str search
final_checkin_data.head()

,0-0,0-1,0-2,0-3,0-4,0-5,0-6,1-0,1-1,1-2,...,8-4,8-5,8-6,9-0,9-1,9-2,9-3,9-4,9-5,9-6
business_id,,,,,,,,,,,,,,,,,,,,,
KO9CpaSPOoqm0iCWm5scmg,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,4.0
oRqBAYtcBYZHXA7G8FlPaA,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6cy2C9aBXUwkrh4bY1DApw,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
D0IB17N66FiyYDCzTlAI4A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HLQGo3EaYVvAv22bONGkIw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [465]:
#Let's try endswith
a = final_checkin_data.columns[final_checkin_data.columns.str.endswith('-0')]

In [466]:
Sunday_data=final_checkin_data[a].head()
Sunday_data
#Since 0-0 is 12am-Sunday and so on, we can see that all of Sunday's data only has been pulled out below

,0-0,1-0,10-0,11-0,12-0,13-0,14-0,15-0,16-0,17-0,...,21-0,22-0,23-0,3-0,4-0,5-0,6-0,7-0,8-0,9-0
business_id,,,,,,,,,,,,,,,,,,,,,
KO9CpaSPOoqm0iCWm5scmg,0.0,0.0,3.0,3.0,20.0,0.0,2.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
oRqBAYtcBYZHXA7G8FlPaA,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6cy2C9aBXUwkrh4bY1DApw,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
D0IB17N66FiyYDCzTlAI4A,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HLQGo3EaYVvAv22bONGkIw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [491]:
Sunday_data.apply((pd.Series),axis=1).sum()

0-0      3.0
1-0      0.0
10-0     5.0
11-0     4.0
12-0    20.0
13-0     0.0
14-0     2.0
15-0     2.0
16-0     0.0
17-0     0.0
18-0     1.0
19-0     1.0
2-0      0.0
20-0     1.0
21-0     1.0
22-0     0.0
23-0     1.0
3-0      0.0
4-0      0.0
5-0      0.0
6-0      0.0
7-0      0.0
8-0      0.0
9-0      0.0
dtype: float64

In [498]:
Sunday2 = Sunday_data.sort_values('12-0',ascending=False)

In [499]:
Sunday2

,0-0,1-0,10-0,11-0,12-0,13-0,14-0,15-0,16-0,17-0,...,21-0,22-0,23-0,3-0,4-0,5-0,6-0,7-0,8-0,9-0
business_id,,,,,,,,,,,,,,,,,,,,,
KO9CpaSPOoqm0iCWm5scmg,0.0,0.0,3.0,3.0,20.0,0.0,2.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
oRqBAYtcBYZHXA7G8FlPaA,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6cy2C9aBXUwkrh4bY1DApw,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
D0IB17N66FiyYDCzTlAI4A,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HLQGo3EaYVvAv22bONGkIw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [500]:
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
plt.scatter(Sunday2['business_id'],Sinday2)